<a href="https://colab.research.google.com/github/Hussnain6/GenerativeAi/blob/main/urdu_roman_poetry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader

# Hyperparameters
EMBEDDING_DIM = 256
HIDDEN_DIM = 512
NUM_LAYERS = 3
BATCH_SIZE = 32
LEARNING_RATE = 0.001
EPOCHS = 15
SEQ_LENGTH = 10

# Load dataset
df = pd.read_csv("Roman-Urdu-Poetry.csv")
poems = df["Poetry"].tolist()
poets = df["Poet"].tolist()

# Tokenize and build vocabulary
words = set()
for poem in poems:
    words.update(poem.split())
word_to_idx = {word: i for i, word in enumerate(words)}
idx_to_word = {i: word for word, i in word_to_idx.items()}
poet_to_idx = {poet: i for i, poet in enumerate(set(poets))}

# Poetry Dataset
class PoetryDataset(Dataset):
    def __init__(self, poems, poets, word_to_idx, poet_to_idx, seq_length=SEQ_LENGTH):
        self.poems = poems
        self.poets = poets
        self.word_to_idx = word_to_idx
        self.poet_to_idx = poet_to_idx
        self.seq_length = seq_length

        self.data = []
        for poem, poet in zip(poems, poets):
            encoded_poem = [word_to_idx[word] for word in poem.split() if word in word_to_idx]
            poet_idx = poet_to_idx[poet]
            for i in range(len(encoded_poem) - seq_length):
                self.data.append((encoded_poem[i:i+seq_length], encoded_poem[i+1:i+seq_length+1], poet_idx))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x, y, poet = self.data[idx]
        return torch.tensor(x), torch.tensor(y), torch.tensor(poet)

# Define LSTM Model
class PoetryLSTM(nn.Module):
    def __init__(self, vocab_size, poet_count, embedding_dim, hidden_dim, num_layers):
        super(PoetryLSTM, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.poet_embedding = nn.Embedding(poet_count, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim * 2, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x, poet):
        word_embeds = self.embedding(x)  # (batch, seq_len, embed_dim)
        poet_embeds = self.poet_embedding(poet).unsqueeze(1).repeat(1, x.size(1), 1)  # (batch, seq_len, embed_dim)
        combined = torch.cat((word_embeds, poet_embeds), dim=2)
        lstm_out, _ = self.lstm(combined)
        out = self.fc(lstm_out)
        return out

# Create Dataset and Dataloader
dataset = PoetryDataset(poems, poets, word_to_idx, poet_to_idx)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

# Model Initialization
vocab_size = len(word_to_idx)
poet_count = len(poet_to_idx)
model = PoetryLSTM(vocab_size, poet_count, EMBEDDING_DIM, HIDDEN_DIM, NUM_LAYERS).to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
criterion = nn.CrossEntropyLoss()

# Training Function
def train_model(model, dataloader, optimizer, criterion, epochs):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for x, y, poet in dataloader:
            x, y, poet = x.to(torch.device("cuda" if torch.cuda.is_available() else "cpu")), y.to(torch.device("cuda" if torch.cuda.is_available() else "cpu")), poet.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))
            optimizer.zero_grad()
            output = model(x, poet)
            loss = criterion(output.view(-1, vocab_size), y.view(-1))
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss / len(dataloader)}")

# Train Model
train_model(model, dataloader, optimizer, criterion, EPOCHS)

Epoch 1/15, Loss: 6.032242679362516
Epoch 2/15, Loss: 3.494838935298606
Epoch 3/15, Loss: 2.0164596887498525
Epoch 4/15, Loss: 1.254115215183949
Epoch 5/15, Loss: 0.8896252694769758
Epoch 6/15, Loss: 0.7300534909719566
Epoch 7/15, Loss: 0.6459786796840393
Epoch 8/15, Loss: 0.5939165720169842
Epoch 9/15, Loss: 0.5573336192314287
Epoch 10/15, Loss: 0.5317638810006151
Epoch 11/15, Loss: 0.5119590276498289
Epoch 12/15, Loss: 0.49578216495543626
Epoch 13/15, Loss: 0.4844878611893032
Epoch 14/15, Loss: 0.4740937258078355
Epoch 15/15, Loss: 0.4667681297117411


In [ ]:
def generate_poetry(model, poet_to_idx, word_to_idx, idx_to_word, max_length=50):
    poet_name = input("Enter poet's name: ")  # User input for poet's name
    start_words = input("Enter starting words for the poem: ")  # User input for start words

    model.eval()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Convert poet name to index
    if poet_name not in poet_to_idx:
        print("Poet not found in dataset.")
        return ""
    poet_idx = torch.tensor([poet_to_idx[poet_name]], device=device)

    # Convert start words to indices
    input_indices = [word_to_idx[word] for word in start_words.split() if word in word_to_idx]
    if not input_indices:
        print("No valid words found in vocabulary.")
        return ""

    input_tensor = torch.tensor(input_indices, dtype=torch.long, device=device).unsqueeze(0)  # Add batch dim

    generated_words = start_words.split()

    for _ in range(max_length):
        with torch.no_grad():
            output = model(input_tensor, poet_idx)  # Forward pass
            predictions = output[:, -1, :]  # Get last token's predictions
            next_word_idx = torch.argmax(predictions, dim=1).item()  # Choose the most probable word

            if next_word_idx in idx_to_word:
                next_word = idx_to_word[next_word_idx]
                generated_words.append(next_word)
                input_tensor = torch.cat((input_tensor, torch.tensor([[next_word_idx]], device=device)), dim=1)
            else:
                break  # Stop if an invalid word is predicted

    return " ".join(generated_words)

generated_poem = generate_poetry(model, poet_to_idx, word_to_idx, idx_to_word)
print("Generated Poem:\n", generated_poem)


Enter poet's name: ahmad-faraz
Enter starting words for the poem: pyaar
Generated Poem:
 pyaar kiran kā dukh nahīñ par dil ye chāhtā hai ki āġhāz tū kare tere baġhair bhī to ġhanīmat hai zindagī ḳhud ko gañvā ke kaun tirī justujū kare ab to ye aarzū hai ki vo zaḳhm khā.iye tā-zindagī ye dil na koī aarzū kare tujh ko bhulā ke dil hai


In [ ]:
torch.save(model.state_dict(), "poetry_lstm.pth")
print("Model saved as poetry_lstm.pth")

Model saved as poetry_lstm.pth


In [ ]:
pip install gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.9/321.9 kB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 119.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 6.3 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [ ]:
import gradio as gr
import torch

# Assuming your model, poet_to_idx, word_to_idx, idx_to_word are already defined

def generate_poetry_gradio(poet_name, start_words):
    # Ensure the model is in evaluation mode
    model.eval()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    if poet_name not in poet_to_idx:
        return "Poet not found in dataset."

    poet_idx = torch.tensor([poet_to_idx[poet_name]], device=device)

    input_indices = [word_to_idx[word] for word in start_words.split() if word in word_to_idx]
    if not input_indices:
        return "No valid words found in vocabulary."

    input_tensor = torch.tensor(input_indices, dtype=torch.long, device=device).unsqueeze(0)

    generated_words = start_words.split()

    for _ in range(50):  # max_length
        with torch.no_grad():
            output = model(input_tensor, poet_idx)
            predictions = output[:, -1, :]
            next_word_idx = torch.argmax(predictions, dim=1).item()

            if next_word_idx in idx_to_word:
                next_word = idx_to_word[next_word_idx]
                generated_words.append(next_word)
                input_tensor = torch.cat((input_tensor, torch.tensor([[next_word_idx]], device=device)), dim=1)
            else:
                break

    return " ".join(generated_words)

# Define Gradio interface
iface = gr.Interface(fn=generate_poetry_gradio,
                     inputs=["text", "text"],
                     outputs="text",
                     live=True,
                     title="Poetry Generator",
                     description="Generate Roman Urdu poetry based on the poet's name and starting words.")

# Launch the interface
iface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://fe804e2858d038fef9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
